## Importação das Bibliotecas

In [2]:
import os
import pandas as pd
import re

## Definicação de variáveis

Define as variáveis relacionada ao arquivo excel de orçamentação

In [3]:
pasta_orcamentos = r'\pasta_orcamentos'
padrao_nome_arq = r'Orçamento-\d+'

#Relativas à informação do cliente e data de orçamentação
lin_min_cd = 0
lin_max_cd = 4
range_cd = 'A:D'
lin_final_cd = lin_max_cd - lin_min_cd

#Relativas à tabela contendo os dados de precificação
lin_min_tb = 3
lin_max_tb = 50
range_tb = 'A:F'
lin_final_tb = lin_max_tb - lin_min_tb

parametro_item_orc = len('margem de contribuição')

## Declara as estruturas de dados

In [4]:
lista_arquivos = []
lista_abas = []
primeiros_registros = {}
ultimos_registros = {}
p_colunas_inf_orc = []
p_registros_inf_orc = []
p_colunas_tab_dados = []
p_registros_tab_dados = []
u_registros_inf_orc = []
u_registros_tab_dados = []
lista_itens_geral = []

## Execução da pesquisa

Pesquisa todos os arquivos presentes na pasta de orçamentação e os relaciona numa lista

In [5]:
for arq in os.listdir(pasta_orcamentos):
    if re.match(padrao_nome_arq, arq):
        lista_arquivos.append(arq)

Em cada arquivo cria dois range de dados: um para a informação de cliente e data e outros para a tabela contendo os dados de precificação.

In [6]:
print(f'Cliente/Data -> Range de {range_cd[0] + str(lin_min_cd)}:{range_cd[2] + str(lin_max_cd)}')
print(f'Tabela de dados -> Range de {range_tb[0] + str(lin_min_tb)}:{range_tb[2] + str(lin_max_tb)}')

Cliente/Data -> Range de A0:D4
Tabela de dados -> Range de A3:F50


Dentro desse range, busca o primeiro valor não nulo, bem como o último valor não nulo dentro da mesma coluna

In [7]:
for arq in lista_arquivos:
    caminho_arquivo = pasta_orcamentos + '\\' + arq

    #Primeiramente, cria um objeto dataframe para cada arquivo excel
    with pd.ExcelFile(caminho_arquivo) as excel:
        #Verifica qual é a última aba do arquivo (versão válida / atual)
        aba_v_valida = max(excel.sheet_names)

        #Nesta últma aba, primeiramente, monta um dataframe baseado no range de dados "Cliente/Data" acima mencionado
        #No qual se espera que esteja a informação do cliente e da data de orçamentação
        df_inf_orc = pd.read_excel(caminho_arquivo, sheet_name = aba_v_valida, usecols = range_cd, skiprows = lin_min_cd, nrows = lin_final_cd, header = None)

        #Itera cada coluna no dataframe criado para procurar o primeiro e o último registro não nulo 
        for col in df_inf_orc.columns:
            idx_primeiro_registro = df_inf_orc[col].first_valid_index()
            
            #Uma vez encontrados os registros, guarda-os em dicionários
            if idx_primeiro_registro is not None:
                vlr_primeiro_registro = df_inf_orc[col][idx_primeiro_registro]
                primeiros_registros[arq] = {'inf_cliente_data': {'nome_coluna': str(col), 'valor': vlr_primeiro_registro}}

                idx_ultimo_registro = df_inf_orc[col].last_valid_index()
                vlr_ultimo_registro = df_inf_orc[col][idx_ultimo_registro]
                ultimos_registros[arq] = {'inf_cliente_data': vlr_ultimo_registro}
                
                break

        #Monta um dataframe baseado no range de dados supracitado no qual se espera que esteja a tabela com os dados de precificação
        df_tab_dados = pd.read_excel(caminho_arquivo, sheet_name = aba_v_valida, usecols = range_tb, skiprows = lin_min_tb, nrows = lin_final_tb, header = None)

        #Itera cada coluna no dataframe criado para procurar o primeiro e o último registro não nulo 
        for col in df_tab_dados.columns:
            idx_primeiro_registro = df_tab_dados[col].first_valid_index()
            
            #Uma vez encontrados os registros, guarda-os em dicionários
            if idx_primeiro_registro is not None:
                vlr_primeiro_registro = df_tab_dados[col][idx_primeiro_registro]
                primeiros_registros[arq].update({'tabela_dados': {'nome_coluna': str(col), 'valor': vlr_primeiro_registro}})

                idx_ultimo_registro = df_tab_dados[col].last_valid_index()
                vlr_ultimo_registro = df_tab_dados[col][idx_ultimo_registro]
                ultimos_registros[arq].update({'tabela_dados': vlr_ultimo_registro})

                break

    lista_itens_arq = [v for v in df_tab_dados.loc[df_tab_dados[col].notna(), col].to_list() if len(v) < parametro_item_orc]

    for i in lista_itens_arq:
        if i not in lista_itens_geral:
            lista_itens_geral.append(i)

## Impressão de resultados

Valores encontrados como primeiros e últimos registros

In [8]:
for v in primeiros_registros.values():

    nome_col_inf_orc = v['inf_cliente_data']['nome_coluna']
    p_vlr_reg_inf_orc = v['inf_cliente_data']['valor']

    if nome_col_inf_orc not in p_colunas_inf_orc:
        p_colunas_inf_orc.append(nome_col_inf_orc)

    if p_vlr_reg_inf_orc not in p_registros_inf_orc:
        p_registros_inf_orc.append(p_vlr_reg_inf_orc)

    nome_col_tab_dados = v['tabela_dados']['nome_coluna']
    p_vlr_reg_tab_dados = v['tabela_dados']['valor']

    if nome_col_tab_dados not in p_colunas_tab_dados:
        p_colunas_tab_dados.append(nome_col_tab_dados)

    if p_vlr_reg_tab_dados not in p_registros_tab_dados:
        p_registros_tab_dados.append(p_vlr_reg_tab_dados)
    
for v in ultimos_registros.values():
    u_vlr_reg_inf_orc = v['inf_cliente_data']

    if u_vlr_reg_inf_orc not in u_registros_inf_orc:
        u_registros_inf_orc.append(u_vlr_reg_inf_orc)

    u_vlr_reg_tab_dados = v['tabela_dados']

    if u_vlr_reg_tab_dados not in u_registros_tab_dados:
        u_registros_tab_dados.append(u_vlr_reg_tab_dados)

In [9]:
print('\033[1;37;40mColunas onde são encontrados os primeiros registros de cliente e data dos orçamentos:\n')

for i in p_colunas_inf_orc:
    print(f'\033[1;32;40m{i}')

print('\n\033[1;37;40mConteúdo dos primeiros registros:\n')

for i in p_registros_inf_orc:
    print(f'\033[1;32;40m{i}')

print('\n\033[1;37;40mConteúdo dos últimos registros:\n')

for i in u_registros_inf_orc:
    print(f'\033[1;32;40m{i}')

print('\n\033[1;37;40m-------------------------------------------------------------\n')
    
print('\033[1;37;40mColunas onde são encontrados os primeiros registros de variáveis de precificação:\n')

for i in p_colunas_tab_dados:
    print(f'\033[1;32;40m{i}')

print('\n\033[1;37;40mConteúdo dos primeiros registros:\n')

for i in p_registros_tab_dados:
    print(f'\033[1;32;40m{i}')  

print('\n\033[1;37;40mConteúdo dos últimos registros:\n')

for i in u_registros_tab_dados:
    print(f'\033[1;32;40m{i}')

Colunas onde são encontrados os primeiros registros de cliente e data dos orçamentos:

0
1
3
2

Conteúdo dos primeiros registros:

Código Cliente:
Cliente:
Cód. Cliente:
*Crédito aprovado / Orçamento válido
*Venda futura

Conteúdo dos últimos registros:

Data:
Dt Orç
*Contato iniciado em 10/02
*Aprovado pela diretoria
Percentual de comissão mantido, cfe orientação da diretoria
Aprovado pelo financeiro
Desconto aprovado pela diretoria
Obs.: percentual de comissão mantido, cfe acordado com o gerente comercial

-------------------------------------------------------------

Colunas onde são encontrados os primeiros registros de variáveis de precificação:

0
1
2

Conteúdo dos primeiros registros:

Preço Bruto
Peço Final
*Venda não aprovada pela Diretoria
*Benefício Fiscal
Receita Bruta
*Aprovado pela Diretoria
Preço Final
Obs.: percentual de comissão mantido, cfe acordado com o gerente comercial

Conteúdo dos últimos registros:

Ebitda
Lucro
*Ebitda muito abaixo do limite estabelecido
*Apro

Texto dos itens encontrados nos arquivos de orçamentação

In [10]:
lista_itens_geral.sort()

print('\033[1;37;40mLista de itens encotrados como variáveis de precificação:\n')

for i in lista_itens_geral:
    print(f'\033[1;32;40m{i}')

Lista de itens encotrados como variáveis de precificação:

*Benefício Fiscal
*Pagamento à vista
Acompanhamento
Assistência Técnica
Avaliação Técnica
Carta Fiança
Comissão
Custo Financeiro
Custos Fixos
Ebitda
Frete
Impostos
Lucro
MC
Margem
Materiais
Mg Contribuição
Outros Custos
Peço Final
Preço Bruto
Preço Final
Preço Líquido
Receita Bruta
Receita Líquida
Seguro
Serviços


Arquivos que destoam do padrão construtivo

In [11]:
inic_padrao_inf_orc = [
    'Código Cliente:',
    'Cliente:',
    'Cód. Cliente:'
]

arquivos_sem_inic_padrao_inf = [k for k, v in primeiros_registros.items() if v['inf_cliente_data']['valor'] not in inic_padrao_inf_orc]

final_padrao_inf_orc = [
    'Data:',
    'Dt Orç'
]

arquivos_sem_final_padrao_inf = [k for k, v in ultimos_registros.items() if v['inf_cliente_data'] not in final_padrao_inf_orc]

inic_padrao_tab_dados = [
    'Preço Bruto',
    'Peço Final',
    'Receita Bruta',
    'Preço Final'
]

arquivos_sem_inic_padrao_tb = [k for k, v in primeiros_registros.items() if v['tabela_dados']['valor'] not in inic_padrao_tab_dados]

final_padrao_tab_dados = [
    'Ebitda', 
    'Lucro'
]

arquivos_sem_final_padrao_tb = [k for k, v in ultimos_registros.items() if v['tabela_dados'] not in final_padrao_tab_dados]

In [12]:

print('\033[1;31;40mInformações de cliente e data do orçamento:\n')

print('\033[1;37;40mArquivos onde o primeiro registro difere dos registros padrões de início:\n')
      
for i in arquivos_sem_inic_padrao_inf:
    print(f'\033[1;32;40m{i}')

print('\033[1;37;40mArquivos onde o último registro difere dos registros padrões de término:\n')

for i in arquivos_sem_final_padrao_inf:
    print(f'\033[1;32;40m{i}')

print('\n--------------------------------------------------------------------------\n')

print('\033[1;31;40mVariáveis de precificação:\n')

print('\033[1;37;40mArquivos onde o primeiro registro difere dos registros padrões de início:\n')
      
for i in arquivos_sem_inic_padrao_tb:
    print(f'\033[1;32;40m{i}')

print('\033[1;37;40mArquivos onde o último registro difere dos registros padrões de término:\n')

for i in arquivos_sem_final_padrao_tb:
    print(f'\033[1;32;40m{i}')

Informações de cliente e data do orçamento:

Arquivos onde o primeiro registro difere dos registros padrões de início:

Orçamento-2097 (Consolidado).xlsx
Orçamento-65 (Consolidado).xlsx
Arquivos onde o último registro difere dos registros padrões de término:

Orçamento-2078 (Consolidado).xlsx
Orçamento-381 (Consolidado).xlsx
Orçamento-5432 (Consolidado).xlsx
Orçamento-5567 (Consolidado).xlsx
Orçamento-5620 (Consolidado).xlsx
Orçamento-5985 (Consolidado).xlsx

--------------------------------------------------------------------------

Variáveis de precificação:

Arquivos onde o primeiro registro difere dos registros padrões de início:

Orçamento-1454.xlsx
Orçamento-25 (Consolidado).xlsx
Orçamento-2922 (Consolidado).xlsx
Orçamento-2952 (Consolidado).xlsx
Orçamento-2966 (Consolidado).xlsx
Orçamento-5985 (Consolidado).xlsx
Arquivos onde o último registro difere dos registros padrões de término:

Orçamento-1454.xlsx
Orçamento-25 (Consolidado).xlsx
Orçamento-2922 (Consolidado).xlsx
Orçamento